In [42]:
#Importando as bibliotecas
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from requests_html import HTML
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.common.exceptions import WebDriverException
from tqdm import tqdm
import sqlite3
import datetime
import os

In [43]:
#Congiruando o driver
options = Options()
options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.add_argument("--log-level=3")
options.add_argument('--no-sandbox')
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [44]:
#Criando listas
Urls_amazon = []

Urls_amazon_more = []

Products_Links = []

Amazon_price = []
Amazon_price_2 = []

Amazon_seller = []
Amazon_seller_2 = []

Amazon_title = []
Amazon_title_2 = []

Amazon_installment_price_full = []
Amazon_installment_price_full_2 = []


internacional_list = []
more_offers_list = []


#Listas Button Sellers
Amazon_Button_Sellers = []
Amazon_Button_Price = []
Amazon_Button_ASIN = []

#Listas More Sellers
Amazon_seller_more = []
Amazon_price_more = []
Amazon_title_more = []
Amazon_ID_more = []

In [45]:
#Listas Button Sellers
Amazon_Button_Sellers = []
Amazon_Button_Price = []
Amazon_Button_ASIN = []

#Listas More Sellers
Amazon_seller_more = []
Amazon_price_more = []
Amazon_title_more = []
Amazon_ID_More = []

In [46]:
driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_103", options=options)

C:\Users\pedro\AppData\Local\Temp\ipykernel_6444\1319122002.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_103", options=options)


In [6]:
#Função para criar os links de busca
def getting_n_creating_amazon(brand):

    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "+")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "+" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.amazon.com.br/s?k=" + df['Urls']

    return df

In [7]:
Df_Products_raw = getting_n_creating_amazon("Wacom")
Df_Products_raw

,Brand,Product_Name,Product_Descrition,Urls,Urls_search
0,wacom,ctl472,ctl 472 - mesa digitalizadora one by wacom peq...,wacom+ctl472,https://www.amazon.com.br/s?k=wacom+ctl472
1,wacom,ctl4100,ctl4100 - mesa digitalizadora wacom intuos sma...,wacom+ctl4100,https://www.amazon.com.br/s?k=wacom+ctl4100
2,wacom,ctl4100wle0,ctl4100wle0 - mesa digitalizadora wacom intuos...,wacom+ctl4100wle0,https://www.amazon.com.br/s?k=wacom+ctl4100wle0
3,wacom,ctl4100wlk0,ctl4100wlk0 - mesa digitalizadora wacom intuos...,wacom+ctl4100wlk0,https://www.amazon.com.br/s?k=wacom+ctl4100wlk0
4,wacom,ctl6100wlk0,ctl6100wlk0 - mesa digitalizadora wacom intuos...,wacom+ctl6100wlk0,https://www.amazon.com.br/s?k=wacom+ctl6100wlk0
5,wacom,dtc133w0a,dtc133w0a - mesa digitalizadora wacom one crea...,wacom+dtc133w0a,https://www.amazon.com.br/s?k=wacom+dtc133w0a
6,wacom,pth460,pth460 - mesa digitalizadora wacom intuos pro ...,wacom+pth460,https://www.amazon.com.br/s?k=wacom+pth460
7,wacom,dtk1660k0a,dtk1660k0a - display interativo wacom cintiq 1...,wacom+dtk1660k0a,https://www.amazon.com.br/s?k=wacom+dtk1660k0a
8,wacom,pth660,pth660 - mesa digitalizadora wacom intuos pro ...,wacom+pth660,https://www.amazon.com.br/s?k=wacom+pth660
9,wacom,ctl6100wle0,ctl6100wle0 - mesa digitalizadora wacom intuos...,wacom+ctl6100wle0,https://www.amazon.com.br/s?k=wacom+ctl6100wle0


In [8]:
Df_Products_raw['Urls_search'][0]

'https://www.amazon.com.br/s?k=wacom+ctl472'

In [48]:
urls = ['https://www.amazon.com.br/s?k=Toshiba+HD+externo+Canvio+Basics+1TB','https://www.amazon.com.br/s?k=HD+Externo+Seagate+Expansion+1TB']

In [49]:
#Função pegando Links
def search_links(url):
    global Urls_amazon

    time.sleep(5)

    driver.get(url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')
    html_obj = HTML(html=html_str)

    Links = [x for x in html_obj.links]
    products_links = [f'https://www.amazon.com.br{x}' for x in Links]

    for link in products_links:
        Urls_amazon.append(link)

    Urls_amazon = [s for s in Urls_amazon if '/dp/' in s]
    Urls_amazon = [s for s in Urls_amazon if not '#customerReviews' in s]

    try:
        Soup = BeautifulSoup(driver.page_source, 'html.parser')
        next_page = "https://www.amazon.com.br" + Soup.find(class_='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator')['href']
        print(next_page)

        search_links(next_page)
    except:
        pass

In [8]:
urls = ['https://www.amazon.com.br/s?k=dji+mavic+mini+2+fly+combo','https://www.amazon.com.br/s?k=dji+mini+se+fly+more+combo']

In [50]:
for url in tqdm(urls):
    search_links(url)

 50%|█████     | 1/2 [00:14<00:14, 14.13s/it]

https://www.amazon.com.br/s?k=HD+Externo+Seagate+Expansion+1TB&page=2&qid=1659327605&ref=sr_pg_1
https://www.amazon.com.br/s?k=HD+Externo+Seagate+Expansion+1TB&page=3&qid=1659327616&ref=sr_pg_2


100%|██████████| 2/2 [00:48<00:00, 24.16s/it]


In [12]:
#Limpando as urls
def clean_link(urls, brand):
    #Criando a conexão com a tabela de exclusão 
    Database_exclusion = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_exclusao"

    #Criando a Query
    query = "SELECT * FROM " + table

    connection = sqlite3.connect(Database_exclusion)

    #Criando o dataset em brando
    df_itens = pd.read_sql_query(query, connection)

    clean_urls = pd.DataFrame()

    clean_urls['Urls_Completas'] = urls
    clean_urls['Urls_limpas'] = clean_urls['Urls_Completas'].str.partition("ref")[0]

    Urls_limpas = clean_urls['Urls_limpas'].tolist()

    for word in df_itens['Words']:
         Urls_limpas = [s for s in Urls_limpas if not word in s]

    Urls_limpas = [s for s in Urls_limpas if 'wacom' in s]

    clean_urls = pd.DataFrame()

    clean_urls['Urls_finais'] = Urls_limpas

    clean_urls['ASIN'] = clean_urls['Urls_finais'].str.partition("/dp/")[2].str.partition("/")[0]

    clean_urls.drop_duplicates(subset='ASIN',inplace=True)
    clean_urls.reset_index(inplace=True, drop=True)
    return clean_urls

In [11]:
New_urls = clean_link(Urls_amazon, 'Motorola')

In [12]:
New_urls

,Urls_finais,ASIN
0,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,B01N807E5P
1,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,B01IRIVFRS
2,https://www.amazon.com.br/Bab%C3%A1-Eletr%C3%B...,B08NFK5QCB
3,https://www.amazon.com.br/Motorola-Bab%C3%A1-E...,B09KMFZN2Y
4,https://www.amazon.com.br/Acess%C3%B3rio-Eletr...,B09CDGLXR5
5,https://www.amazon.com.br/Bab%C3%A1-Eletr%C3%B...,B08QFCNPXJ
6,https://www.amazon.com.br/Eletronica-Motorola-...,B084QZ7JPF
7,https://www.amazon.com.br/Motorola-Baba-Eletro...,B08FVXB74T
8,https://www.amazon.com.br/Eletronica-Digital-M...,B0731J6K8M
9,https://www.amazon.com.br/Motorola-MBP-36S-SC-...,B00NSF730O


In [51]:
def search_atributes(url):
    #Tempo para não haver o bloqueio
    time.sleep(5)

    #Entrando dentro do site com o driver
    driver.get(url)

    #Tempo para carregar
    time.sleep(10)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    #Criando o Soup
    soup = BeautifulSoup(html_str, 'html.parser')

    #Fazendo o try do nome do vendedor
    try:
        seller = soup.find(id='sellerProfileTriggerId').text
        seller_2 = soup.find("a", attrs={"id":'sellerProfileTriggerId'}).text
        Amazon_seller.append(seller)
        Amazon_seller_2.append(seller_2)
    except:
        Amazon_seller.append("Erro")
        Amazon_seller_2.append("Erro")

    #Fazendo o try do preço do produto a vista
    try:
        Div_Price = soup.find('div', attrs={"data-feature-name":"corePrice"})
        price_2 = Div_Price.find(class_='a-offscreen').text
        Amazon_price_2.append(price_2)
    except:
        try:
            Amazon_price_2.append(soup.find(class_='a-button-text')['href'])
        except:
            Amazon_price_2.append("Erro")

    #Pegando o título do produto
    try:
        title = soup.find(id='productTitle').text
        Amazon_title.append(title)
    except:
        Amazon_title.append('Erro')

    #Pegando o internacional
    try:
        soup.find('img', attrs={'data-a-hires':'https://images-na.ssl-images-amazon.com/images/G/32/foreignseller/Foreign_Seller_Badge_v2._CB403622375_.png'})
        internacional_list.append("Internacional")
    except:
        internacional_list.append("Nacional")

    #Fazendo o try para pegar o preço da parcela
    try:
        installment = soup.find(class_='best-offer-name a-text-bold').text
        Amazon_installment_price_full.append(installment)
        
        Div_installment = soup.find('div', attrs={'id':'installmentCalculator_feature_div'})
        Installment_2 = Div_installment.find(class_='best-offer-name a-text-bold').text
        Amazon_installment_price_full_2.append(Installment_2)
    except:
        Amazon_installment_price_full.append("0")
        Amazon_installment_price_full_2.append("Erro")

    #Fazendo o try para ver se tem mais ofertas
    try:
        Main_Div_More_offers = soup.find('div', attrs={"id":"olpLinkWidget_feature_div"})
        Div_More_offers = Main_Div_More_offers.find('div', attrs={'class':'a-section olp-link-widget'})
        Div_More_offers_text = Div_More_offers.find('div', attrs={'class':'olp-text-box'}).text
        more_offers_list.append(Div_More_offers_text)
    except:
        more_offers_list.append("Comparar outras 0 ofertas")


In [52]:
for url in tqdm(Urls_amazon):
    search_atributes(url)

100%|██████████| 131/131 [37:38<00:00, 17.24s/it]


In [53]:
print(len(Amazon_title))
print(len(Amazon_price_2))
print(len(Amazon_seller_2))
print(len(more_offers_list))
print(len(Amazon_installment_price_full_2))
print(len(internacional_list))

131
131
131
131
131
131


In [54]:
def dataset_amazon(url, sellers, preco, titulo, more_url):
    # Criando o DataFrame
    Dataset_amazon = pd.DataFrame()

    #
    Hoje = pd.to_datetime('today', errors='ignore').date()

    # Colocando os dados
    Dataset_amazon['URL'] = url

    Dataset_amazon['DATE'] = Hoje

    Dataset_amazon['MARKETPLACE'] = "AMAZON"

    #Arrumando a coluna de sellers
    Dataset_amazon['SELLER'] = sellers
    Dataset_amazon['SELLER'] = Dataset_amazon['SELLER'].str.replace("Erro","Amazon", regex=False)

    #Arrumando o preço
    Dataset_amazon['PRICE'] = preco
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace(".","", regex=True)
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace("R$","", regex=False)
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace(",",".", regex=True)

    #Arrumando os valores de installment
    Dataset_amazon['INSTALLMENT FULL'] = Amazon_installment_price_full
    Dataset_amazon['PARCEL'] = Dataset_amazon['INSTALLMENT FULL'].str.extract('(\d+)')
    Dataset_amazon['PARCEL'] = Dataset_amazon['PARCEL'].astype("int")
    Dataset_amazon['parcel_price_bruto'] = \
    Dataset_amazon['INSTALLMENT FULL'].str.partition("R$")[2].str.partition(" ")[2].str.partition(" ")[0]
    Dataset_amazon['Installment3'] = Dataset_amazon['parcel_price_bruto'].str.extract('(\d+)')
    Dataset_amazon['parcel_price_bruto'] = \
    Dataset_amazon['INSTALLMENT FULL'].str.partition("R$")[2].str.partition(" ")[2].str.partition(" ")[0].str.partition(",")[2]
    Dataset_amazon['Installment4'] = Dataset_amazon['parcel_price_bruto'].str.extract('(\d+)')
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['Installment3'] + "." + Dataset_amazon['Installment4']
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['INSTALLMENT'].astype("float")
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['INSTALLMENT'].fillna(0)
    Dataset_amazon['INSTALLMENT_PAYMENT'] = Dataset_amazon['PARCEL'] * Dataset_amazon['INSTALLMENT']


    Dataset_amazon['ID'] = Dataset_amazon['URL'].str.partition('/dp/')[2].str.partition('/')[0]
    Dataset_amazon['PRODUCT'] = titulo
    Dataset_amazon['INTERNACIONAL'] = internacional_list

    #Arrumando valores de mais sellers
    Dataset_amazon['MORE'] = more_url
    Dataset_amazon['MORE'] = Dataset_amazon['MORE'].str.partition("outras ")[2].str.partition(" ofertas")[0]

    Dataset_amazon['MORE'] = Dataset_amazon['MORE'].astype('int')


    Dataset_amazon = Dataset_amazon.drop(columns=["INSTALLMENT FULL", "Installment3", "Installment4", "parcel_price_bruto"])


    #Colocando na ordem correta
    Dataset_amazon = Dataset_amazon[['DATE', 'URL', 'MARKETPLACE', 'SELLER', 'PRICE', 'PARCEL', 'INSTALLMENT', 'INSTALLMENT_PAYMENT', 'ID', 'PRODUCT','INTERNACIONAL', 'MORE']]

    return Dataset_amazon

In [55]:
Df_final = dataset_amazon(Urls_amazon, Amazon_seller_2, Amazon_price_2, Amazon_title, more_offers_list)

In [56]:
Df_final=  Df_final.reset_index(drop=True)

In [57]:
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
0,2022-08-01,https://www.amazon.com.br/Toshiba-Canvio-Advan...,AMAZON,Amazon,/gp/offer-listing/B08JKGHHHQ/ref=dp_olp_unknow...,0,0.00,0.0,B08JKGHHHQ,TOSHIBA Canvio Advance Disco rígido ex...,Internacional,0
1,2022-08-01,https://www.amazon.com.br/Toshiba-HDTB330XK3CA...,AMAZON,Amazon,/gp/offer-listing/B00T5ME88Y/ref=dp_olp_unknow...,0,0.00,0.0,B00T5ME88Y,Toshiba Canvio Basics Portátil Externo...,Internacional,0
2,2022-08-01,https://www.amazon.com.br/Externo-Toshiba-Canv...,AMAZON,OctoShop BR,871.04,10,87.14,871.4,B08JKJ9XLN,HD Externo Portátil Toshiba 2TB Canvio...,Internacional,4
3,2022-08-01,https://www.amazon.com.br/Toshiba-Port%C3%A1ti...,AMAZON,Tendinfo,998.90,10,99.89,998.9,B079H83D53,HD Externo Portátil Toshiba Canvio Adv...,Internacional,0
4,2022-08-01,https://www.amazon.com.br/Externo-Port%C3%A1ti...,AMAZON,Tendinfo,1525.00,10,152.50,1525.0,B07KQX5VHY,HD Externo Portátil Toshiba Canvio Adv...,Internacional,0
...,...,...,...,...,...,...,...,...,...,...,...,...
126,2022-08-01,https://www.amazon.com.br/Seagate-externo-foto...,AMAZON,Triplenet Pricing INC,3150.05,10,315.05,3150.5,B093WZHS1G,"Seagate One Touch Hub, 8 TB, disco ríg...",Internacional,3
127,2022-08-01,https://www.amazon.com.br/Seagate-Disco-r%C3%A...,AMAZON,Triplenet Pricing INC,2883.87,10,288.45,2884.5,B093BWLBYY,Seagate Disco rígido externo USB 3.0 d...,Internacional,3
128,2022-08-01,https://www.amazon.com.br/Seagate-externo-serv...,AMAZON,Amazon,/gp/offer-listing/B08KSJ7Z5F/ref=dp_olp_unknow...,0,0.00,0.0,B08KSJ7Z5F,Seagate One Touch 4 TB Disco externo H...,Internacional,0
129,2022-08-01,https://www.amazon.com.br/Externo-1TB-Toshiba-...,AMAZON,OctoShop BR,662.29,10,66.31,663.1,B079H7VYY6,HD Externo Portátil Toshiba Canvio Adv...,Internacional,2


In [21]:
Df_final = Df_final[Df_final['PRICE'].str.len() < 10]
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
0,2022-08-01,https://www.amazon.com.br/Flash-Godox-Tt685-Ca...,AMAZON,Fênix Foto E Vídeo,999.00,10,99.90,999.00,B07DB6H8LV,Flash Godox Tt685 N Ttl Para Camera Ca...,Internacional,2
1,2022-08-01,https://www.amazon.com.br/Flash-Speedlite-Godo...,AMAZON,Amazon,515.00,10,51.50,515.00,B07664WHHZ,GODOX Flash Speedlite TT560 II com Rad...,Internacional,9
2,2022-08-01,https://www.amazon.com.br/Flash-Godox-TT685S-C...,AMAZON,Pixel Equipamentos,939.99,10,94.08,940.80,B01GHLNRL4,Godox TT685S Thinklite TTL Flash para ...,Internacional,2
3,2022-08-01,https://www.amazon.com.br/V860III-S-disparador...,AMAZON,More Shot,1259.00,10,125.90,1259.00,B0B1H39JWM,Godox V860III-S c/ X2T-S Flash de disp...,Internacional,0
4,2022-08-01,https://www.amazon.com.br/TT685II-F-TT685IIF-W...,AMAZON,YouTin,796.00,10,79.60,796.00,B09QYSY119,Godox Flash de câmera TT685II-F TT685I...,Internacional,0
5,2022-08-01,https://www.amazon.com.br/V860II-S-gatilho-rec...,AMAZON,More Shot,1199.00,10,119.90,1199.00,B0B1H5TQDJ,Godox V860II-S c/ X2T-S Kit de gatilho...,Internacional,0
6,2022-08-01,https://www.amazon.com.br/Flash-Godox-Thinklit...,AMAZON,FOTO CENTRO,995.00,10,99.50,995.00,B01GNZMZ26,Godox TT685C ThinkLite C de alta veloc...,Internacional,3
7,2022-08-01,https://www.amazon.com.br/V860III-S-Speedlite-...,AMAZON,More Shot,1259.00,10,125.90,1259.00,B09ZDRY3M1,Godox Flash V860III-S para câmera Sony...,Internacional,0
8,2022-08-01,https://www.amazon.com.br/Flash-Tocha-Estudio-...,AMAZON,FOTO CENTRO,1399.00,10,139.90,1399.00,B07DB67F3Q,Flash SK400II para Estúdio Fotográfico...,Internacional,7
9,2022-08-01,https://www.amazon.com.br/Panasonic-Speedlight...,AMAZON,More Shot,689.00,10,68.90,689.00,B09LCY9TP9,Godox TT685II-O TT685IIO Flash de câme...,Internacional,0


In [58]:
#Criar a condicional da coluna More
Df_button_sellers = Df_final[Df_final['PRICE'].str.len() > 10]
Df_button_sellers

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
0,2022-08-01,https://www.amazon.com.br/Toshiba-Canvio-Advan...,AMAZON,Amazon,/gp/offer-listing/B08JKGHHHQ/ref=dp_olp_unknow...,0,0.0,0.0,B08JKGHHHQ,TOSHIBA Canvio Advance Disco rígido ex...,Internacional,0
1,2022-08-01,https://www.amazon.com.br/Toshiba-HDTB330XK3CA...,AMAZON,Amazon,/gp/offer-listing/B00T5ME88Y/ref=dp_olp_unknow...,0,0.0,0.0,B00T5ME88Y,Toshiba Canvio Basics Portátil Externo...,Internacional,0
8,2022-08-01,https://www.amazon.com.br/Externo-Port%C3%A1ti...,AMAZON,Amazon,/gp/offer-listing/B00N2S6ZUQ/ref=dp_olp_unknow...,0,0.0,0.0,B00N2S6ZUQ,HD Externo Portátil Toshiba Canvio Bas...,Internacional,0
12,2022-08-01,https://www.amazon.com.br/Externo-Portatil-Tos...,AMAZON,Amazon,/gp/offer-listing/B00N2S73UC/ref=dp_olp_unknow...,0,0.0,0.0,B00N2S73UC,TOSHIBA Canvio Basics Disco rígido por...,Internacional,0
34,2022-08-01,https://www.amazon.com.br/seagate-externo-port...,AMAZON,Amazon,/gp/offer-listing/B0711V6Q3K/ref=dp_olp_unknow...,0,0.0,0.0,B0711V6Q3K,HD seagate externo portátil game drive...,Internacional,0
35,2022-08-01,https://www.amazon.com.br/Seagate-Expansion-ST...,AMAZON,Amazon,/gp/offer-listing/B07VBPG5HZ/ref=dp_olp_unknow...,0,0.0,0.0,B07VBPG5HZ,Seagate Expansion STEA5000402 disco rí...,Internacional,0
41,2022-08-01,https://www.amazon.com.br/Seagate-FireCuda-Gam...,AMAZON,Amazon,/gp/offer-listing/B08Y1QYWR5/ref=dp_olp_unknow...,0,0.0,0.0,B08Y1QYWR5,Seagate FireCuda Gaming Hub Disco rígi...,Internacional,0
52,2022-08-01,https://www.amazon.com.br/Desktop-Seagate-Back...,AMAZON,Amazon,/gp/offer-listing/B01K4241D0/ref=dp_olp_unknow...,0,0.0,0.0,B01K4241D0,"Hd Desktop Backup Plus Hub 6Tb, Seagat...",Internacional,0
67,2022-08-01,https://www.amazon.com.br/Port%C3%A1til-Seagat...,AMAZON,Amazon,/gp/offer-listing/B076H6TZ7L/ref=dp_olp_unknow...,0,0.0,0.0,B076H6TZ7L,HD Seagate Externo Portátil Game Drive...,Internacional,0
74,2022-08-01,https://www.amazon.com.br/Seagate-port%C3%A1ti...,AMAZON,Amazon,/gp/offer-listing/B08JDS8HM7/ref=dp_olp_unknow...,0,0.0,0.0,B08JDS8HM7,"Seagate One Touch, disco rígido extern...",Internacional,0


In [59]:
def Get_Button_Sellers(ASIN):
    global Amazon_Button_Sellers

    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=1&experienceId=aodAjaxMain"

    driver.get(new_url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base'):
        Amazon_Button_Sellers.append(seller.text)


    Amazon_Button_Sellers =  [s for s in Amazon_Button_Sellers if not 'avaliações' in s]
    Amazon_Button_Sellers =  [s for s in Amazon_Button_Sellers if not ' Amazon.com.br ' in s]
    Amazon_Button_Sellers =  [s for s in Amazon_Button_Sellers if not 'avaliação' in s]
    Amazon_Button_Sellers =  [s for s in Amazon_Button_Sellers if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen'):
        Amazon_Button_Price.append(price.text)
        Amazon_Button_ASIN.append(ASIN)

In [60]:
for item in tqdm(Df_button_sellers['ID']):
    Get_Button_Sellers(item)

100%|██████████| 13/13 [02:18<00:00, 10.63s/it]


In [61]:
print(len(Amazon_Button_Sellers))
print(len(Amazon_Button_Price))
print(len(Amazon_Button_ASIN))

19
19
19


In [62]:
Dataframe_Button = pd.DataFrame()
Dataframe_Button['ID'] = Amazon_Button_ASIN

Dataframe_Button['DATE'] = pd.to_datetime('today', errors='ignore').date()

Dataframe_Button['MARKETPLACE'] = 'AMAZON'

Dataframe_Button['SELLER'] = Amazon_Button_Sellers

Dataframe_Button['PRICE'] = Amazon_Button_Price
Dataframe_Button['PRICE'] = Dataframe_Button['PRICE'].str.replace(".","", regex=True)
Dataframe_Button['PRICE'] = Dataframe_Button['PRICE'].str.replace("R$","", regex=False)
Dataframe_Button['PRICE'] = Dataframe_Button['PRICE'].str.replace(",",".", regex=True)
Dataframe_Button['PRICE'] = Dataframe_Button['PRICE'].astype('float')

Dataframe_Button['PARCEL'] = 10

Dataframe_Button['INSTALLMENT'] = Dataframe_Button['PRICE'] / Dataframe_Button['PARCEL']

Dataframe_Button['INSTALLMENT_PAYMENT'] = Dataframe_Button['PRICE'] * Dataframe_Button['PARCEL']

Dataframe_Button['INTERNACIONAL'] = 'ERRO'

url_names = []
for id in Dataframe_Button['ID']:
    url_names.append(Df_final.loc[Df_final['ID'] == id,'URL'].values[0])

products_names = []
for id in Dataframe_Button['ID']:
    products_names.append(Df_final.loc[Df_final['ID'] == id,'PRODUCT'].values[0])

Dataframe_Button['URL'] = url_names
Dataframe_Button['PRODUCT'] = products_names

Dataframe_Button

,ID,DATE,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,INTERNACIONAL,URL,PRODUCT
0,B08JKGHHHQ,2022-08-01,AMAZON,BUY SUPPLY,874.00,10,87.400,8740.0,ERRO,https://www.amazon.com.br/Toshiba-Canvio-Advan...,TOSHIBA Canvio Advance Disco rígido ex...
1,B00T5ME88Y,2022-08-01,AMAZON,BUY SUPPLY,1541.00,10,154.100,15410.0,ERRO,https://www.amazon.com.br/Toshiba-HDTB330XK3CA...,Toshiba Canvio Basics Portátil Externo...
2,B00N2S6ZUQ,2022-08-01,AMAZON,BUY SUPPLY,747.00,10,74.700,7470.0,ERRO,https://www.amazon.com.br/Externo-Port%C3%A1ti...,HD Externo Portátil Toshiba Canvio Bas...
3,B00N2S73UC,2022-08-01,AMAZON,BUY SUPPLY,733.00,10,73.300,7330.0,ERRO,https://www.amazon.com.br/Externo-Portatil-Tos...,TOSHIBA Canvio Basics Disco rígido por...
4,B0711V6Q3K,2022-08-01,AMAZON,Tendinfo,1349.00,10,134.900,13490.0,ERRO,https://www.amazon.com.br/seagate-externo-port...,HD seagate externo portátil game drive...
5,B0711V6Q3K,2022-08-01,AMAZON,Tendinfo,1349.80,10,134.980,13498.0,ERRO,https://www.amazon.com.br/seagate-externo-port...,HD seagate externo portátil game drive...
6,B07VBPG5HZ,2022-08-01,AMAZON,BRVENDAS,2000.00,10,200.000,20000.0,ERRO,https://www.amazon.com.br/Seagate-Expansion-ST...,Seagate Expansion STEA5000402 disco rí...
7,B08Y1QYWR5,2022-08-01,AMAZON,ComegaCity,6873.08,10,687.308,68730.8,ERRO,https://www.amazon.com.br/Seagate-FireCuda-Gam...,Seagate FireCuda Gaming Hub Disco rígi...
8,B01K4241D0,2022-08-01,AMAZON,Mundo dos Computadores,1159.00,10,115.900,11590.0,ERRO,https://www.amazon.com.br/Desktop-Seagate-Back...,"Hd Desktop Backup Plus Hub 6Tb, Seagat..."
9,B01K4241D0,2022-08-01,AMAZON,Mundo dos Computadores,1165.40,10,116.540,11654.0,ERRO,https://www.amazon.com.br/Desktop-Seagate-Back...,"Hd Desktop Backup Plus Hub 6Tb, Seagat..."


In [63]:
print(Df_final.shape[0])
print(Dataframe_Button.shape[0])

131
19


In [64]:
Df_final = Df_final[Df_final['PRICE'].str.len() < 10]
print(Df_final.shape[0])

118


In [65]:
Df_final_with_button = pd.concat([Df_final, Dataframe_Button])
print(Df_final_with_button.shape[0])

137


In [66]:
Df_final_with_button = Df_final_with_button[Df_final_with_button['PRICE'] != 'Erro']

In [67]:
Df_final_with_button['PRICE'] = Df_final_with_button['PRICE'].astype("float")

In [68]:
Df_final_with_button = Df_final_with_button[Df_final_with_button['PRICE'] > 100.99]

In [69]:
Df_final_with_button['MORE'] = Df_final_with_button['MORE'].fillna(0.0)
Df_final_with_button['MORE'] = Df_final_with_button['MORE'].astype('int')

In [70]:
Df_final_with_button

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
2,2022-08-01,https://www.amazon.com.br/Externo-Toshiba-Canv...,AMAZON,OctoShop BR,871.04,10,87.140,871.4,B08JKJ9XLN,HD Externo Portátil Toshiba 2TB Canvio...,Internacional,4
3,2022-08-01,https://www.amazon.com.br/Toshiba-Port%C3%A1ti...,AMAZON,Tendinfo,998.90,10,99.890,998.9,B079H83D53,HD Externo Portátil Toshiba Canvio Adv...,Internacional,0
4,2022-08-01,https://www.amazon.com.br/Externo-Port%C3%A1ti...,AMAZON,Tendinfo,1525.00,10,152.500,1525.0,B07KQX5VHY,HD Externo Portátil Toshiba Canvio Adv...,Internacional,0
5,2022-08-01,https://www.amazon.com.br/Western-Digital-WDBU...,AMAZON,Webeletron,419.99,10,42.080,420.8,B06W55K9N6,Western Digital WDBU6Y0020BBK Externo ...,Internacional,15
6,2022-08-01,https://www.amazon.com.br/externo-Seagate-Expa...,AMAZON,LOJA DA MADA,439.00,10,43.900,439.0,B07Q3SPNJ2,HD externo Seagate Expansion 2TB portá...,Internacional,15
...,...,...,...,...,...,...,...,...,...,...,...,...
14,2022-08-01,https://www.amazon.com.br/Port%C3%A1til-Seagat...,AMAZON,WAZ - Fanáticos por Tecnologia,1294.11,10,129.411,12941.1,B076H6TZ7L,HD Seagate Externo Portátil Game Drive...,ERRO,0
15,2022-08-01,https://www.amazon.com.br/Seagate-port%C3%A1ti...,AMAZON,Triplenet Pricing INC,805.43,10,80.543,8054.3,B08JDS8HM7,"Seagate One Touch, disco rígido extern...",ERRO,0
16,2022-08-01,https://www.amazon.com.br/Seagate-Expans%C3%A3...,AMAZON,ComegaCity,1967.63,10,196.763,19676.3,B093BVXYSV,Seagate Expansão STKP4000400 Disco ríg...,ERRO,0
17,2022-08-01,https://www.amazon.com.br/Seagate-Disco-r%C3%A...,AMAZON,ComegaCity,5521.48,10,552.148,55214.8,B093BWYJ9Q,Seagate Disco rígido externo USB 3.0 d...,ERRO,0


In [71]:
Df_more_offers = Df_final[Df_final['MORE'] != 0]
Df_more_offers

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
2,2022-08-01,https://www.amazon.com.br/Externo-Toshiba-Canv...,AMAZON,OctoShop BR,871.04,10,87.14,871.40,B08JKJ9XLN,HD Externo Portátil Toshiba 2TB Canvio...,Internacional,4
5,2022-08-01,https://www.amazon.com.br/Western-Digital-WDBU...,AMAZON,Webeletron,419.99,10,42.08,420.80,B06W55K9N6,Western Digital WDBU6Y0020BBK Externo ...,Internacional,15
6,2022-08-01,https://www.amazon.com.br/externo-Seagate-Expa...,AMAZON,LOJA DA MADA,439.00,10,43.90,439.00,B07Q3SPNJ2,HD externo Seagate Expansion 2TB portá...,Internacional,15
7,2022-08-01,https://www.amazon.com.br/EXT-SEAGATE-500GB-US...,AMAZON,HEROWEN,233.78,7,33.44,234.08,B00UNA1IPI,Seagate HD EXT. 500GB USB 3.0 EXPANSIO...,Internacional,10
9,2022-08-01,https://www.amazon.com.br/HD-1TB-Toshiba-HDTC9...,AMAZON,OctoShop BR,662.29,10,66.31,663.10,B079H8PZPV,HD Externo Portátil Toshiba Canvio Adv...,Internacional,2
...,...,...,...,...,...,...,...,...,...,...,...,...
124,2022-08-01,https://www.amazon.com.br/Seagate-Disco-r%C3%A...,AMAZON,Amazin Group,1082.71,10,108.28,1082.80,B08Y1QRTBM,Seagate Disco rígido externo portátil ...,Internacional,6
125,2022-08-01,https://www.amazon.com.br/Externo-Toshiba-Canv...,AMAZON,OctoShop BR,795.43,10,79.57,795.70,B08JKHPV1V,HD Externo Portátil Toshiba 2TB Canvio...,Internacional,5
126,2022-08-01,https://www.amazon.com.br/Seagate-externo-foto...,AMAZON,Triplenet Pricing INC,3150.05,10,315.05,3150.50,B093WZHS1G,"Seagate One Touch Hub, 8 TB, disco ríg...",Internacional,3
127,2022-08-01,https://www.amazon.com.br/Seagate-Disco-r%C3%A...,AMAZON,Triplenet Pricing INC,2883.87,10,288.45,2884.50,B093BWLBYY,Seagate Disco rígido externo USB 3.0 d...,Internacional,3


In [72]:
def search_more_offers_1(ASIN):
    global Amazon_seller_more
    
    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=1&experienceId=aodAjaxMain"

    driver.get(new_url)
    
    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base')[4:]:
        Amazon_seller_more.append(seller.text)


    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen')[2:]:
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [73]:
def search_more_offers_2(ASIN):
    global Amazon_seller_more

    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=2&experienceId=aodAjaxMain"

    driver.get(new_url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base'):
        Amazon_seller_more.append(seller.text)


    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen'):
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [74]:
def search_more_offers_3(ASIN):
    global Amazon_seller_more

    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=3&experienceId=aodAjaxMain"

    driver.get(new_url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base'):
        Amazon_seller_more.append(seller.text)


    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen'):
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [75]:
def search_more_offers_4(ASIN):
    global Amazon_seller_more

    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=4&experienceId=aodAjaxMain"

    driver.get(new_url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base'):
        Amazon_seller_more.append(seller.text)


    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen'):
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [76]:
def search_more_offers_5(ASIN):
    global Amazon_seller_more

    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=5&experienceId=aodAjaxMain"

    driver.get(new_url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base'):
        Amazon_seller_more.append(seller.text)


    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen'):
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [77]:
for id,more in zip(Df_more_offers.ID, Df_more_offers.MORE):
    if more < 10:
        search_more_offers_1(id)
    elif (more > 10) and (more < 20):
        search_more_offers_1(id)
        search_more_offers_2(id)
    elif (more > 20) and (more < 30):
        search_more_offers_1(id)
        search_more_offers_2(id)
        search_more_offers_3(id)
    elif (more > 30) and (more < 40):
        search_more_offers_1(id)
        search_more_offers_2(id)
        search_more_offers_3(id)
        search_more_offers_4(id)
    elif (more > 40) and (more < 50):
        search_more_offers_1(id)
        search_more_offers_2(id)
        search_more_offers_3(id)
        search_more_offers_4(id)
        search_more_offers_5(id)

In [78]:
print(len(Amazon_seller_more))
print(len(Amazon_ID_More))
print(len(Amazon_price_more))

531
531
531


In [40]:
new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + "B079NWPZTF" + "&pageno=1&experienceId=aodAjaxMain"

driver.get(new_url)

time.sleep(5)

body_el = driver.find_element(By.CSS_SELECTOR, 'body')
html_str = body_el.get_attribute('innerHTML')

Soup = BeautifulSoup(html_str, 'html.parser')

for seller in Soup.find_all(class_='a-size-small a-color-base')[4:]:
    Amazon_seller_more.append(seller.text)


Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

for price in Soup.find_all(class_='a-offscreen')[2:]:
    Amazon_price_more.append(price.text)
    Amazon_ID_More.append('B079NWPZTF')

KeyboardInterrupt: 

In [ ]:
for name in Amazon_seller_more:
    print(name)

In [ ]:
for price in Amazon_price_more:
    print(price)

In [ ]:
print(len(Amazon_seller_more))
print(len(Amazon_ID_More))
print(len(Amazon_price_more))

In [ ]:
Amazon_seller_more = []
Amazon_price_more = []

In [ ]:
Amazon_seller_more

In [ ]:
Amazon_seller_more.append("Erro")

In [ ]:
Dataframe = pd.DataFrame()

Dataframe['ID'] = Amazon_ID_More
Dataframe['SELLER'] = Amazon_seller_more
Dataframe['PRICE'] = Amazon_price_more

In [ ]:
Dataframe

In [ ]:
Amazon_seller_more

In [ ]:
Amazon_price_more

In [79]:
Dataframe_More = pd.DataFrame()
Dataframe_More['ID'] = Amazon_ID_More

Dataframe_More['DATE'] = pd.to_datetime('today', errors='ignore').date()

Dataframe_More['MARKETPLACE'] = 'AMAZON'

Dataframe_More['SELLER'] = Amazon_seller_more

Dataframe_More['PRICE'] = Amazon_price_more
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].str.replace(".","", regex=True)
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].str.replace("R$","", regex=False)
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].str.replace(",",".", regex=True)
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].astype('float')

Dataframe_More['PARCEL'] = 10

Dataframe_More['INSTALLMENT'] = Dataframe_More['PRICE'] / Dataframe_More['PARCEL']

Dataframe_More['INSTALLMENT_PAYMENT'] = Dataframe_More['PRICE'] * Dataframe_More['PARCEL']

Dataframe_More['INTERNACIONAL'] = 'ERRO'

url_names = []
for id in Dataframe_More['ID']:
    url_names.append(Df_final.loc[Df_final['ID'] == id,'URL'].values[0])

products_names = []
for id in Dataframe_More['ID']:
    products_names.append(Df_final.loc[Df_final['ID'] == id,'PRODUCT'].values[0])

Dataframe_More['URL'] = url_names
Dataframe_More['PRODUCT'] = products_names

Dataframe_More

,ID,DATE,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,INTERNACIONAL,URL,PRODUCT
0,B08JKJ9XLN,2022-08-01,AMAZON,PedroAMSeller,839.00,10,83.900,8390.0,ERRO,https://www.amazon.com.br/Externo-Toshiba-Canv...,HD Externo Portátil Toshiba 2TB Canvio...
1,B08JKJ9XLN,2022-08-01,AMAZON,BUY SUPPLY,923.00,10,92.300,9230.0,ERRO,https://www.amazon.com.br/Externo-Toshiba-Canv...,HD Externo Portátil Toshiba 2TB Canvio...
2,B08JKJ9XLN,2022-08-01,AMAZON,Tendinfo,1099.00,10,109.900,10990.0,ERRO,https://www.amazon.com.br/Externo-Toshiba-Canv...,HD Externo Portátil Toshiba 2TB Canvio...
3,B06W55K9N6,2022-08-01,AMAZON,LOJA FENIX,429.00,10,42.900,4290.0,ERRO,https://www.amazon.com.br/Western-Digital-WDBU...,Western Digital WDBU6Y0020BBK Externo ...
4,B06W55K9N6,2022-08-01,AMAZON,NANU SHOP,431.00,10,43.100,4310.0,ERRO,https://www.amazon.com.br/Western-Digital-WDBU...,Western Digital WDBU6Y0020BBK Externo ...
...,...,...,...,...,...,...,...,...,...,...,...
526,B093WZHS1G,2022-08-01,AMAZON,ComegaCity,3150.04,10,315.004,31500.4,ERRO,https://www.amazon.com.br/Seagate-externo-foto...,"Seagate One Touch Hub, 8 TB, disco ríg..."
527,B093WZHS1G,2022-08-01,AMAZON,Rover Mall,3038.83,10,303.883,30388.3,ERRO,https://www.amazon.com.br/Seagate-externo-foto...,"Seagate One Touch Hub, 8 TB, disco ríg..."
528,B093BWLBYY,2022-08-01,AMAZON,ComegaCity,2883.86,10,288.386,28838.6,ERRO,https://www.amazon.com.br/Seagate-Disco-r%C3%A...,Seagate Disco rígido externo USB 3.0 d...
529,B093BWLBYY,2022-08-01,AMAZON,Rover Mall,2777.65,10,277.765,27776.5,ERRO,https://www.amazon.com.br/Seagate-Disco-r%C3%A...,Seagate Disco rígido externo USB 3.0 d...


In [80]:
Df_True_Final = pd.concat([Df_final_with_button, Dataframe_More])
Df_True_Final = Df_True_Final.sort_values(by='ID')

In [81]:
Df_True_Final = Df_True_Final[Df_True_Final['PRICE'] > 100.00]

In [82]:
Df_True_Final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
362,2022-08-01,https://www.amazon.com.br/Seagate-na8hll4n-HD-...,AMAZON,NANU SHOP,668.99,10,66.899,6689.9,B0065KEVA4,HD Externo Seagate 4tb,ERRO,NaN
66,2022-08-01,https://www.amazon.com.br/Seagate-na8hll4n-HD-...,AMAZON,LOJA DA MADA,669.99,10,67.080,670.8,B0065KEVA4,HD Externo Seagate 4tb,Internacional,13.0
373,2022-08-01,https://www.amazon.com.br/Seagate-na8hll4n-HD-...,AMAZON,Acessoriotec,1500.00,10,150.000,15000.0,B0065KEVA4,HD Externo Seagate 4tb,ERRO,NaN
372,2022-08-01,https://www.amazon.com.br/Seagate-na8hll4n-HD-...,AMAZON,Mundo dos Computadores,719.54,10,71.954,7195.4,B0065KEVA4,HD Externo Seagate 4tb,ERRO,NaN
370,2022-08-01,https://www.amazon.com.br/Seagate-na8hll4n-HD-...,AMAZON,Style Vip,750.00,10,75.000,7500.0,B0065KEVA4,HD Externo Seagate 4tb,ERRO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
529,2022-08-01,https://www.amazon.com.br/Seagate-Disco-r%C3%A...,AMAZON,Rover Mall,2777.65,10,277.765,27776.5,B093BWLBYY,Seagate Disco rígido externo USB 3.0 d...,ERRO,NaN
17,2022-08-01,https://www.amazon.com.br/Seagate-Disco-r%C3%A...,AMAZON,ComegaCity,5521.48,10,552.148,55214.8,B093BWYJ9Q,Seagate Disco rígido externo USB 3.0 d...,ERRO,0.0
526,2022-08-01,https://www.amazon.com.br/Seagate-externo-foto...,AMAZON,ComegaCity,3150.04,10,315.004,31500.4,B093WZHS1G,"Seagate One Touch Hub, 8 TB, disco ríg...",ERRO,NaN
527,2022-08-01,https://www.amazon.com.br/Seagate-externo-foto...,AMAZON,Rover Mall,3038.83,10,303.883,30388.3,B093WZHS1G,"Seagate One Touch Hub, 8 TB, disco ríg...",ERRO,NaN


In [83]:
Df_True_Final.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Software_Artemis\Data\Brand_Search_Test\Amazon_Toshiba.xlsx", index=False)

In [46]:
 Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
0,2022-06-10,https://www.amazon.com.br/Digitalizadora-gr%C3...,AMAZON,MilleniumShop,469.95,10,47.04,470.40,B087NNZSR8,Mesa Digitalizadora XP-PEN Star G960S ...,Internacional,4
2,2022-06-10,https://www.amazon.com.br/Folha-Textura-Macia-...,AMAZON,Amazon,151.72,5,30.36,151.80,B07195N1C6,Folha De Textura Wacom Macia - Tamanho...,Internacional,0
3,2022-06-10,https://www.amazon.com.br/Display-Interativo-C...,AMAZON,Amazon,6390.00,6,6.32,37.92,B07L77GTTY,"Display Interativo Cintiq 16, Wacom, D...",Internacional,3
5,2022-06-10,https://www.amazon.com.br/Mesa-Digitalizadora-...,AMAZON,NETVENDAS,219.99,7,31.47,220.29,B07S1RR3FR,Mesa Digitalizadora One by Wacom CTL47...,Internacional,8
6,2022-06-10,https://www.amazon.com.br/Display-Interativo-W...,AMAZON,Amazon,4037.49,10,403.83,4038.30,B094489GDW,"Display Interativo Wacom One, DTC133W0...",Internacional,2
7,2022-06-10,https://www.amazon.com.br/Display-Interativo-W...,AMAZON,Amazon,2889.99,3,3.11,9.33,B082LZXQ6B,"Display Interativo Wacom One, DTC133W0...",Internacional,9
8,2022-06-10,https://www.amazon.com.br/Caneta-Wacom-Tablets...,AMAZON,Amazon,579.90,644,644.33,414948.52,B079J2FSQ7,Mesa Digitalizadora Wacom Bluetooth Pe...,Internacional,16
9,2022-06-10,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,INPOWER,1329.00,10,132.90,1329.00,B079NWPZTF,Mesa Digitalizadora Wacom Média Blueto...,Internacional,9
10,2022-06-10,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,Amazon,4716.36,10,471.69,4716.90,B01MTZ9CCD,Mesa Digitalizadora Wacom Intuos Pro L...,Internacional,5
11,2022-06-10,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,W-TECH IMPORTS,5997.00,10,599.70,5997.00,B01MSXN9M5,Mesa Digitalizadora Intuos Pro Paper G...,Internacional,4


In [37]:
Df_final.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\AMAZON.xlsx", index=False)
Dataframe_Button.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\AMAZON_BUTTON.xlsx", index=False)
Dataframe_More.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\AMAZON_MORE.xlsx", index=False)

In [35]:
Df_final_copy = Df_final[Df_final['PRICE'].str.len() < 10]
Df_final_copy = Df_final_copy[Df_final_copy['MORE'] == 0]
Df_final_copy

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
6,2022-06-06,https://www.amazon.com.br/C%C3%82MERA-GOPRO-HE...,AMAZON,GRUPO MAVERICK,3099.99,10,310.08,3100.80,B09LZ9655F,CÂMERA GOPRO HERO 8 BLACK - POWER BUND...,Internacional,0
14,2022-06-06,https://www.amazon.com.br/C%C3%82MERA-GOPRO-HE...,AMAZON,GRUPO MAVERICK,3099.99,10,310.08,3100.80,B09MG3XKZJ,CÂMERA GOPRO HERO 8 BLACK – SPECIAL BU...,Internacional,0
15,2022-06-06,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...,AMAZON,LIVE IMPORTS,3099.00,10,309.90,3099.00,B08R2XCM9K,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional,0
35,2022-06-06,https://www.amazon.com.br/fengny-Substitui%C3%...,AMAZON,kokorest,183.40,6,30.60,183.60,B09ZPB2QW3,fengny Substituição de controle remoto...,Internacional,0
39,2022-06-06,https://www.amazon.com.br/Acess%C3%B3rios-para...,AMAZON,FUNPro,275.00,9,30.60,275.40,B086M7FK58,Kit de Acessórios para GoPro Hero 8 Bl...,Internacional,0
40,2022-06-06,https://www.amazon.com.br/Zwbfu-acess%C3%B3rio...,AMAZON,Jeneza,177.99,5,35.63,178.15,B09GJMPY6R,Zwbfu Kit de acessórios para câmera de...,Internacional,0
44,2022-06-06,https://www.amazon.com.br/Insta360-imperme%C3%...,AMAZON,EVO Gimbals,3636.86,10,363.74,3637.40,B08M3B9754,Insta360 Câmera de ação impermeável ON...,Internacional,0
48,2022-06-06,https://www.amazon.com.br/Conjunto-acess%C3%B3...,AMAZON,Minimini,75.00,2,37.50,75.00,B082X8HPLL,Conjunto de acessórios para câmera Mob...,Internacional,0
49,2022-06-06,https://www.amazon.com.br/Kit-50-Acess%C3%B3ri...,AMAZON,Amazon,342.14,10,34.25,342.50,B07LFLJBW4,Kit 50 Em 1 De Acessórios Para Gopro ...,Internacional,0
50,2022-06-06,https://www.amazon.com.br/PORTA-USB-PARA-PRONL...,AMAZON,GRUPO MAVERICK,299.99,9,33.35,300.15,B09BBJTF8R,PORTA USB PARA USO PRONLONGADO HERO 9 ...,Internacional,0


In [36]:
Df_final_copy

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
6,2022-06-06,https://www.amazon.com.br/C%C3%82MERA-GOPRO-HE...,AMAZON,GRUPO MAVERICK,3099.99,10,310.08,3100.80,B09LZ9655F,CÂMERA GOPRO HERO 8 BLACK - POWER BUND...,Internacional,0
14,2022-06-06,https://www.amazon.com.br/C%C3%82MERA-GOPRO-HE...,AMAZON,GRUPO MAVERICK,3099.99,10,310.08,3100.80,B09MG3XKZJ,CÂMERA GOPRO HERO 8 BLACK – SPECIAL BU...,Internacional,0
15,2022-06-06,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...,AMAZON,LIVE IMPORTS,3099.00,10,309.90,3099.00,B08R2XCM9K,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional,0
35,2022-06-06,https://www.amazon.com.br/fengny-Substitui%C3%...,AMAZON,kokorest,183.40,6,30.60,183.60,B09ZPB2QW3,fengny Substituição de controle remoto...,Internacional,0
39,2022-06-06,https://www.amazon.com.br/Acess%C3%B3rios-para...,AMAZON,FUNPro,275.00,9,30.60,275.40,B086M7FK58,Kit de Acessórios para GoPro Hero 8 Bl...,Internacional,0
40,2022-06-06,https://www.amazon.com.br/Zwbfu-acess%C3%B3rio...,AMAZON,Jeneza,177.99,5,35.63,178.15,B09GJMPY6R,Zwbfu Kit de acessórios para câmera de...,Internacional,0
44,2022-06-06,https://www.amazon.com.br/Insta360-imperme%C3%...,AMAZON,EVO Gimbals,3636.86,10,363.74,3637.40,B08M3B9754,Insta360 Câmera de ação impermeável ON...,Internacional,0
48,2022-06-06,https://www.amazon.com.br/Conjunto-acess%C3%B3...,AMAZON,Minimini,75.00,2,37.50,75.00,B082X8HPLL,Conjunto de acessórios para câmera Mob...,Internacional,0
49,2022-06-06,https://www.amazon.com.br/Kit-50-Acess%C3%B3ri...,AMAZON,Amazon,342.14,10,34.25,342.50,B07LFLJBW4,Kit 50 Em 1 De Acessórios Para Gopro ...,Internacional,0
50,2022-06-06,https://www.amazon.com.br/PORTA-USB-PARA-PRONL...,AMAZON,GRUPO MAVERICK,299.99,9,33.35,300.15,B09BBJTF8R,PORTA USB PARA USO PRONLONGADO HERO 9 ...,Internacional,0


In [130]:
url_names = []
for id in Dataframe_More['ID']:
    url_names.append(Df_final.loc[Df_final['ID'] == id,'URL'])

In [132]:
len(url_names)

21

In [114]:
Df_final.loc[Df_final['ID'] == Dataframe_More['ID'],'URL']

ValueError: Can only compare identically-labeled Series objects

In [57]:
Df_teste = pd.concat([Df_final, Dataframe_Button])
Df_teste = Df_teste.sort_values(by='ID')

In [58]:
Df_teste

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
44,2022-05-31,https://www.amazon.com.br/GoPole-Alcan%C3%A7a-...,AMAZON,FUNPro,208.40,6,34.750,208.50,B00B3YFDBS,GoPole Alcança de extensão de 14 a 101...,Internacional,0.0
22,2022-05-31,https://www.amazon.com.br/GoPro-Sistema-HERO3-...,AMAZON,LIG Celular,189.9,10,18.990,1899.00,B00J9RO4U2,GoPro Sistema Dual Hero para HERO3+ (c...,ERRO,NaN
14,2022-05-31,https://www.amazon.com.br/GOPOLE-Reach-Mini-ex...,AMAZON,Ami Ventures Inc.,1034.0,10,103.400,10340.00,B01C397F1C,GOPOLE Reach Mini – Haste de extensão ...,ERRO,NaN
62,2022-05-31,https://www.amazon.com.br/transporte-acess%C3%...,AMAZON,wecome2021,158.00,5,31.600,158.00,B01H6OP92G,Estojo de transporte para GoPro Hero(2...,Internacional,0.0
26,2022-05-31,https://www.amazon.com.br/GoPro-Porta-Lateral-...,AMAZON,GRUPO MAVERICK,189.99,10,18.999,1899.90,B01L2CPKOK,"Porta Lateral de Reposição para Hero6,...",ERRO,NaN
20,2022-05-31,https://www.amazon.com.br/C%C3%A2mera-Digital-...,AMAZON,Tech With Expert,4602.0,10,460.200,46020.00,B0792MJLNM,"Câmera Fusion 360, GoPro, Preto",ERRO,NaN
19,2022-05-31,https://www.amazon.com.br/C%C3%A2mera-Digital-...,AMAZON,Porto Seguro Store,3190.0,10,319.000,31900.00,B0792MJLNM,"Câmera Fusion 360, GoPro, Preto",ERRO,NaN
13,2022-05-31,https://www.amazon.com.br/C%C3%A2mera-GoPro-MA...,AMAZON,LIVE IMPORTS,3499.0,10,349.900,34990.00,B07BPVPV14,"Câmera GoPro MAX 360, Preto",ERRO,NaN
12,2022-05-31,https://www.amazon.com.br/C%C3%A2mera-GoPro-MA...,AMAZON,GRUPO MAVERICK,3485.21,10,348.521,34852.10,B07BPVPV14,"Câmera GoPro MAX 360, Preto",ERRO,NaN
11,2022-05-31,https://www.amazon.com.br/Camera-Digital-12-1M...,AMAZON,João e Maria Shop,2574.05,10,257.405,25740.50,B07GSVDFTQ,Camera Digital GoPro Hero 7 Black Ultr...,ERRO,NaN


In [37]:
Df_Final_Final = pd.concat([Df_final_copy, Dataframe_Button])
Df_Final_Final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
6,2022-06-06,https://www.amazon.com.br/C%C3%82MERA-GOPRO-HE...,AMAZON,GRUPO MAVERICK,3099.99,10,310.080,3100.80,B09LZ9655F,CÂMERA GOPRO HERO 8 BLACK - POWER BUND...,Internacional,0.0
14,2022-06-06,https://www.amazon.com.br/C%C3%82MERA-GOPRO-HE...,AMAZON,GRUPO MAVERICK,3099.99,10,310.080,3100.80,B09MG3XKZJ,CÂMERA GOPRO HERO 8 BLACK – SPECIAL BU...,Internacional,0.0
15,2022-06-06,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...,AMAZON,LIVE IMPORTS,3099.00,10,309.900,3099.00,B08R2XCM9K,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional,0.0
35,2022-06-06,https://www.amazon.com.br/fengny-Substitui%C3%...,AMAZON,kokorest,183.40,6,30.600,183.60,B09ZPB2QW3,fengny Substituição de controle remoto...,Internacional,0.0
39,2022-06-06,https://www.amazon.com.br/Acess%C3%B3rios-para...,AMAZON,FUNPro,275.00,9,30.600,275.40,B086M7FK58,Kit de Acessórios para GoPro Hero 8 Bl...,Internacional,0.0
40,2022-06-06,https://www.amazon.com.br/Zwbfu-acess%C3%B3rio...,AMAZON,Jeneza,177.99,5,35.630,178.15,B09GJMPY6R,Zwbfu Kit de acessórios para câmera de...,Internacional,0.0
44,2022-06-06,https://www.amazon.com.br/Insta360-imperme%C3%...,AMAZON,EVO Gimbals,3636.86,10,363.740,3637.40,B08M3B9754,Insta360 Câmera de ação impermeável ON...,Internacional,0.0
48,2022-06-06,https://www.amazon.com.br/Conjunto-acess%C3%B3...,AMAZON,Minimini,75.00,2,37.500,75.00,B082X8HPLL,Conjunto de acessórios para câmera Mob...,Internacional,0.0
49,2022-06-06,https://www.amazon.com.br/Kit-50-Acess%C3%B3ri...,AMAZON,Amazon,342.14,10,34.250,342.50,B07LFLJBW4,Kit 50 Em 1 De Acessórios Para Gopro ...,Internacional,0.0
50,2022-06-06,https://www.amazon.com.br/PORTA-USB-PARA-PRONL...,AMAZON,GRUPO MAVERICK,299.99,9,33.350,300.15,B09BBJTF8R,PORTA USB PARA USO PRONLONGADO HERO 9 ...,Internacional,0.0


In [38]:
Df_Final_Final = pd.concat([Df_final_copy, Dataframe_Button, Dataframe_More])
Df_Final_Final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
6,2022-06-06,https://www.amazon.com.br/C%C3%82MERA-GOPRO-HE...,AMAZON,GRUPO MAVERICK,3099.99,10,310.080,3100.8,B09LZ9655F,CÂMERA GOPRO HERO 8 BLACK - POWER BUND...,Internacional,0.0
14,2022-06-06,https://www.amazon.com.br/C%C3%82MERA-GOPRO-HE...,AMAZON,GRUPO MAVERICK,3099.99,10,310.080,3100.8,B09MG3XKZJ,CÂMERA GOPRO HERO 8 BLACK – SPECIAL BU...,Internacional,0.0
15,2022-06-06,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...,AMAZON,LIVE IMPORTS,3099.00,10,309.900,3099.0,B08R2XCM9K,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional,0.0
35,2022-06-06,https://www.amazon.com.br/fengny-Substitui%C3%...,AMAZON,kokorest,183.40,6,30.600,183.6,B09ZPB2QW3,fengny Substituição de controle remoto...,Internacional,0.0
39,2022-06-06,https://www.amazon.com.br/Acess%C3%B3rios-para...,AMAZON,FUNPro,275.00,9,30.600,275.4,B086M7FK58,Kit de Acessórios para GoPro Hero 8 Bl...,Internacional,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
154,2022-06-06,https://www.amazon.com.br/Dji-Osmo-Pocket-2-Co...,AMAZON,Lucas Lapa PhotoPro,5999.0,10,599.900,59990.0,B08HWCCS6S,Dji Osmo Pocket 2 Combo,ERRO,NaN
155,2022-06-06,https://www.amazon.com.br/Lateral-Reposi%C3%A7...,AMAZON,FUNPro,91.9,10,9.190,919.0,B07GDG1D6H,GoPro Acessório de câmera de substitui...,ERRO,NaN
156,2022-06-06,https://www.amazon.com.br/Lateral-Reposi%C3%A7...,AMAZON,GRUPO MAVERICK,149.99,10,14.999,1499.9,B07GDG1D6H,GoPro Acessório de câmera de substitui...,ERRO,NaN
157,2022-06-06,https://www.amazon.com.br/Insta360-C%C3%A2mera...,AMAZON,Imperioshopping,4621.9,10,462.190,46219.0,B08YRZP1QN,Insta360 Câmera ONE X2 360 com tela se...,ERRO,NaN


In [39]:
Df_Final_Final.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\AMAZON_FINAL_GOPRO.xlsx", index=False)

In [59]:
Df_final_mesmo = pd.concat([Df_teste, Dataframe_More])
Df_final_mesmo

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
44,2022-05-31,https://www.amazon.com.br/GoPole-Alcan%C3%A7a-...,AMAZON,FUNPro,208.40,6,34.750,208.5,B00B3YFDBS,GoPole Alcança de extensão de 14 a 101...,Internacional,0.0
22,2022-05-31,https://www.amazon.com.br/GoPro-Sistema-HERO3-...,AMAZON,LIG Celular,189.9,10,18.990,1899.0,B00J9RO4U2,GoPro Sistema Dual Hero para HERO3+ (c...,ERRO,NaN
14,2022-05-31,https://www.amazon.com.br/GOPOLE-Reach-Mini-ex...,AMAZON,Ami Ventures Inc.,1034.0,10,103.400,10340.0,B01C397F1C,GOPOLE Reach Mini – Haste de extensão ...,ERRO,NaN
62,2022-05-31,https://www.amazon.com.br/transporte-acess%C3%...,AMAZON,wecome2021,158.00,5,31.600,158.0,B01H6OP92G,Estojo de transporte para GoPro Hero(2...,Internacional,0.0
26,2022-05-31,https://www.amazon.com.br/GoPro-Porta-Lateral-...,AMAZON,GRUPO MAVERICK,189.99,10,18.999,1899.9,B01L2CPKOK,"Porta Lateral de Reposição para Hero6,...",ERRO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
144,2022-05-31,https://www.amazon.com.br/Dji-Osmo-Pocket-2-Co...,AMAZON,BalbinoShop,3899.99,10,389.999,38999.9,B08HWCCS6S,Dji Osmo Pocket 2 Combo,ERRO,NaN
145,2022-05-31,https://www.amazon.com.br/Dji-Osmo-Pocket-2-Co...,AMAZON,GearNPC,5000.0,10,500.000,50000.0,B08HWCCS6S,Dji Osmo Pocket 2 Combo,ERRO,NaN
146,2022-05-31,https://www.amazon.com.br/Dji-Osmo-Pocket-2-Co...,AMAZON,eFacini,5072.9,10,507.290,50729.0,B08HWCCS6S,Dji Osmo Pocket 2 Combo,ERRO,NaN
147,2022-05-31,https://www.amazon.com.br/Dji-Osmo-Pocket-2-Co...,AMAZON,Lucas Lapa PhotoPro,5999.0,10,599.900,59990.0,B08HWCCS6S,Dji Osmo Pocket 2 Combo,ERRO,NaN


In [60]:
Df_final_mesmo.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\AMAZON_FINAL_GOPRO.xlsx", index=False)

In [105]:
Dataframe_More['DATE'] = pd.to_datetime('today', errors='ignore').date()
#Dataframe_More['URL'] = [Df_final.loc[Df_final['ID'] == Dataframe_More['ID'], 'URL']]

ValueError: Can only compare identically-labeled Series objects

In [117]:
Dataframe_More

,ID,Seller,Price,DATE,URL
0,B07WSRXMS8,LIVE IMPORTS,"R$ 2.299,00",2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...
1,B07WSRXMS8,King Pro St,"R$ 2.289,00",2022-05-21,https://www.amazon.com.br/GoPro-Frontal-HyperS...
2,B07WSRXMS8,GRUPO MAVERICK,"R$ 2.599,00",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-HE...
3,B09D15SFMQ,LIVE IMPORTS,"R$ 3.207,25",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...
4,B09D15SFMQ,King Pro St,"R$ 3.219,00",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-Fr...
5,B09D15SFMQ,GRUPO MAVERICK,"R$ 4.299,99",2022-05-21,https://www.amazon.com.br/C%C3%82MERA-GOPRO-BL...
6,B09D15SFMQ,MeuDome,"R$ 4.799,00",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Digital-...
7,B08R2XCM9K,EOS IMPORT,"R$ 2.999,00",2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...
8,B08KXN6BNH,King Pro St,"R$ 2.599,00",2022-05-21,NaN
9,B08KXN6BNH,King Pro St,"R$ 2.739,00",2022-05-21,NaN


In [63]:
#Df_final = Df_final.drop(columns=['MORE'])
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL
0,2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...,AMAZON,EOS IMPORT,2299.00,10,229.90,2299.0,B07WSRXMS8,Gopro Hero 8 Black 4k à prova D'água C...,Internacional
1,2022-05-21,https://www.amazon.com.br/GoPro-Frontal-HyperS...,AMAZON,EOS IMPORT,3207.25,10,320.77,3207.7,B09D15SFMQ,Câmera GoPro HERO10 Black à Prova com ...,Internacional
2,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-HE...,AMAZON,Amazon,4688.20,10,468.82,4688.2,B093JKGDJZ,Câmera GoPro HERO9 Black - Kit Trip Pr...,Internacional
3,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...,AMAZON,EOS IMPORT,3000.00,10,300.00,3000.0,B08R2XCM9K,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional
4,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-Fr...,AMAZON,EOS IMPORT,2540.73,10,254.10,2541.0,B08KXN6BNH,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional
5,2022-05-21,https://www.amazon.com.br/C%C3%82MERA-GOPRO-BL...,AMAZON,EOS IMPORT,2599.00,10,259.90,2599.0,B08ZSVP9T7,CÂMERA GOPRO HERO 8 BLACK ESSENCIAL BU...,Internacional
6,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Digital-...,AMAZON,EOS IMPORT,3799.00,10,379.90,3799.0,B087ZP2GW4,Câmera Digital GoPro Max 360o - CHDHZ-...,Internacional
7,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,HuaYe Tech,1982.90,10,198.29,1982.9,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,Internacional


In [39]:
Df_teste.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Motorola_Amazon.xlsx", index=False)

In [33]:
Amazon_seller_more =  [s for s in Amazon_seller_more if len(s) > 1]
print(len(Amazon_seller_more))

187


In [39]:
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Política de devolução' in s]
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Apagar tudo' in s]

print(len(Amazon_seller_more))

0


In [37]:
Amazon_seller_more

[' Tudo à Beça',
 'Política de devolução',
 ' Gigantec',
 'Política de devolução',
 ' INFOSHOP.',
 'Política de devolução',
 ' Thaun Shop',
 'Política de devolução',
 ' Tendinfo',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 '',
 'Apagar tudo',
 ' O MUNDO PARA VOCÊ',
 'Política de devolução',
 ' ZAPATA STORE SP',
 'Política de devolução',
 ' OFICINA DOS BITS',
 'Política de devolução',
 ' Syma Solutions',
 'Política de devolução',
 ' Conect Magazine',
 'Política de devolução',
 ' IBYTE',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 ' MAGIC WORLD ELETRONICOS',
 'Política de devolução',
 ' REX INFORMATICA',
 'Política de devolução',
 ' Tudo à Beça',
 'Política de devolução',
 ' W-TECH IMPORTS',
 'Política de devolução',
 ' Kadri Tecnologia',
 'Política de devolução',
 ' E-SPOT',
 'Política de devolução',
 ' Tendinfo',
 'Política de devolução',
 '',
 'Apagar tudo',
 '

In [33]:
Amazon_ID_More = []

In [ ]:
Amazon_seller_more = []
Amazon_price_more = []


In [ ]:
Amazon_seller_more

In [ ]:
Amazon_price_more

In [ ]:

new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + "B09D15SFMQ" + "&pageno=1&experienceId=aodAjaxMain"

driver.get(new_url)

time.sleep(5)

body_el = driver.find_element(By.CSS_SELECTOR, 'body')
html_str = body_el.get_attribute('innerHTML')

Soup = BeautifulSoup(html_str, 'html.parser')

for seller in Soup.find_all(class_='a-size-small a-link-normal')[4:]:
    print(seller.text)

for price in Soup.find_all(class_='a-offscreen')[2:]:
    print(price.text)


In [ ]:
for url in Df_more_offers['ID']:
    time.sleep(3)

    search = "https://www.amazon.com.br/gp/product/ajax/?asin=" + url + "&pageno=1&experienceId=aodAjaxMain"

    driver.get(search)
    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    print(search)

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-link-normal'):
        print(seller.text)

In [ ]:
#Teste gráfico 
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
plt.boxplot(x=Df_final['PRICE'], vert=False)
plt.show()

In [ ]:
df_products = Df_final[Df_final['PRICE'] > 800]

In [ ]:
df_products